# Problem Analysis Workshop 1


Group members:

- Jiho Jun - 9080800
- Vishnu Sivaraj - 9025320
- Edwin Lopez - 9055061


## Question:

### Predicting hospital readmission for diabetes patients within 30 days


In [ ]:
# Install necessary packages

# !pip install -q kaggle pandas

In [1]:
# import necessary libraries

import os, json, pathlib
from getpass import getpass

In [3]:
# Get Kaggle token from user input
print("Paste the contents of kaggle.json. Example: {\"username\":\"your_name\",\"key\":\"abcd...\"}")

# get token string
token_str = getpass("kaggle.json content: ")

Paste the contents of kaggle.json. Example: {"username":"your_name","key":"abcd..."}


In [4]:
# save to ~/.kaggle/kaggle.json
home = pathlib.Path.home()

# make .kaggle directory
kdir = home/".kaggle"

# create directory if it doesn't exist
kdir.mkdir(exist_ok=True)

# write the token string to kaggle.json file
(kdir/"kaggle.json").write_text(token_str)

# set file permissions to read/write for user only
os.chmod(kdir/"kaggle.json", 0o600)

# confirm saved
print("Saved to", kdir/"kaggle.json")

Saved to C:\Users\Ed\.kaggle\kaggle.json


In [5]:
# import KaggleApi and authenticate
from kaggle.api.kaggle_api_extended import KaggleApi
from pathlib import Path

# authenticate
api = KaggleApi()
api.authenticate()

In [6]:
# path to download directory
out_dir = Path("data/diabetes")

# create output directory if it doesn't exist
out_dir.mkdir(parents=True, exist_ok=True)

# dataset to download
dataset = "brandao/diabetes"

print("Downloading", dataset)

# download and unzip
api.dataset_download_files(dataset, path=str(out_dir), unzip=True)

print("Complete.")

print("Files:")

# list downloaded files
for p in sorted(out_dir.rglob("*")):
    if p.is_file():
        print("-", p.relative_to(out_dir))

Dataset URL: https://www.kaggle.com/datasets/brandao/diabetes
Complete.
Files:
- description.pdf
- diabetic_data.csv


In [ ]:
# import pandas
import pandas as pd

# path to the CSV file
file_path = "data/diabetes/diabetic_data.csv"

# Load the CSV into a DataFrame
df = pd.read_csv(file_path)

# Show basic info and first few rows
print(df.shape)

# data types of each column
print(df.dtypes)

# first few rows
df.head()


# Data Cleansing


This data needs cleansing before analysis:

1. Handle missing and unknown values

   - Many categorical fields use "?" or "Unknown/Invalid" instead of nulls.

2. Simplify target variable

   - The field **readmitted** has three categories:  
     "NO"

     ">30" (readmitted after 30 days)

     "<30" (readmitted within 30 days).

     Collapse into binary classification:

     1 = readmitted within 30 days ("<30")

     0 = not readmitted ("NO" or ">30").

3. Encode categorical variables

   - Convert age from intervals (e.g. [70-80)) to numeric midpoints (e.g. 75).

4. Reduce dimensionality of diagnoses

   - There are hundreds of unique codes, which create sparse features ( diag_1, diag_2, diag_3 )
   - There are hundreds of unique codes, which create sparse features.  
     Approach: Group into broader disease categories (circulatory, respiratory, digestive, diabetes-related, etc.).

5. Address class imbalance
   - Address Class Imbalance. In this data, about 11% of cases are readmitted within 30 days.
   - Undersample majority class.
